<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/main/user_based_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config

# Configuration

In [1]:
#@title Download necessary libraries
!pip install pyspark
!pip install -U -q PyDrive 
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=eb69251f9172f5cef910b4cf1e6280bc9ef17184c3ba890ffa91112bf8d50cb5
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [2]:
#@title Imports
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly

import pyspark
from pyspark.sql import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml.linalg import SparseVector, DenseVector

from tqdm.notebook import tqdm
import time
import gc

from google.colab import drive

In [3]:
#@title Set up variables
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/drive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [4]:
#@title Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [253]:
drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


## Setup ngrok

In [6]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 28.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=6163541fb8457ad398d0ddd35b1b40deff378d6a0cfbb9488b3d0e09f5591bd2
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [7]:
!ngrok authtoken 2NVN8kdoOnMVtlDGGWtwsbT5M3Q_2EJv2HE77FEXkz978Qtnq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [8]:
from pyngrok import ngrok

# Open a ngrok tunnel on the port 4050 where Spark is running
port = '4050'
public_url = ngrok.connect(port).public_url

In [9]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://d1a4-34-86-240-214.ngrok-free.app" -> "http://127.0.0.1:4050"


In [10]:
#@title Check if everything is ok
spark, sc._conf.getAll()


(<pyspark.sql.session.SparkSession at 0x7f0eab2a14e0>,
 [('spark.app.id', 'local-1684224759796'),
  ('spark.executor.extraJavaOptions',
   '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false -XX:+UseG1GC'),
  ('spark.app.name', 'PySparkTutorial')

# Data acquisition

In [11]:

song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

audio_features_schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("type", StringType(), True),
    StructField("id", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("track_href", StringType(), True),
    StructField("analysis_url", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("time_signature", IntegerType(), True)
])


In [12]:
playlist_df = spark.read.schema(playlist_schema).json(DATASET_FILE, multiLine=True)
slice_df = spark.read.schema(playlist_schema).json(SMALL_SLICE_FLIE, multiLine=True)
# slice_df = spark.read.schema(playlist_schema).json(LITTLE_SLICE_FILE, multiLine=True)
audio_df = spark.read.schema(audio_features_schema).json(SPLITTED_SLICE_AUDIO_FEATURES, multiLine=True) #has less songs than expected

In [13]:
# slice_df.select("tracks").first()

In [14]:
# slice_df.show()

# User-Based Collaborative Filtering
Note: The users are the playlists, the items are the songs and the ratings are 0 if the song is not in the playlist, 1 otherwise.

We have to define a function $sim(u,v)$ that defines the similarity between two users based on their ratings.

We represent the ratings $r_u \in \mathbb{R}^n$ as the $n$ dimensional vector that represents the ratings of the user $u$, where $n$ is the number of total songs in the dataset.

As the similarity function we can use Jaccard similarity.
\begin{equation}
sim(u,v) = J(r_u, r_v) = \frac{|r_u \cap r_v|}{|r_u \cup r_v|}
\end{equation}

Jaccard similarity ignores rating values, but we don't care here since the ratings are binary. In case of discrete value ratings we can use cosine similarity, or better pearson's correlation.

Done that, and defined as ${U^k}$ the neighborhood of $u$ ($k$ most similar users to $u$), we define the set of items rated by $u$'s neighborhood as

\begin{equation}
I^k = \{i \in I : \mathbf{r_{u,i}} \downarrow \land u \in U^k\}
\end{equation}

The rating for the item $i$ to the user $u$ will just be $\mathbf{r_u[i]}$.

In [15]:
RATING_VECTOR_FILE_PATH = os.path.join(SAVED_DFS_PATH, "playlist_rating_df.parquet")
PLAYLIST_EMBEDDINGS = os.path.join(SAVED_DFS_PATH, "playlist_embeddings.parquet")
FULL_PLAYLIST_EMBEDDINGS = os.path.join(SAVED_DFS_PATH, "full_playlist_embeddings.parquet")

In [16]:
def dense_to_sparse(dense: DenseVector) -> SparseVector:
  nonzero_indices = np.nonzero(np.array(dense))[0]
  nonzero_values = np.array(dense)[nonzero_indices]
  sparse_vector = SparseVector(len(dense), nonzero_indices.tolist(), nonzero_values.tolist())
  return sparse_vector

In [17]:
def get_all_songs(playlist_df: DataFrame, set_in_playlist: bool = False) -> DataFrame:
   all_songs = playlist_df.select(explode("tracks.track_uri").alias("track_uri")).distinct()
   if set_in_playlist:
     all_songs = all_songs.withColumn("in_playlist", lit(1))
   return all_songs
  
def get_songs_info(playlist_df: DataFrame, set_in_playlist: bool = False) -> DataFrame:
   all_songs = playlist_df.select(explode("tracks")).select("col.*").drop("pos").distinct()
   if set_in_playlist:
     all_songs = all_songs.withColumn("in_playlist", lit(1))
   return all_songs

# songs_df = get_all_songs(slice_df)
# songs_df.createOrReplaceTempView("SONGS")

# songs_df = spark.sql("""
# SELECT 
#     row_number() OVER (
#         PARTITION BY '' 
#         ORDER BY '' 
#     ) as pos,
#     *
# FROM 
#     SONGS
# """)

# songs_df = songs_df.sort("track_uri")

# songs_info_df = get_songs_info(slice_df)

# songs_info_df = songs_info_df.join(songs_df, "track_uri", "left")

# RATING_VECTOR_LENGTH = songs_df.count()

In [18]:
songs_info_df = get_songs_info(slice_df)
songs_info_df.createOrReplaceTempView("SONGS_INFO")

songs_info_df = spark.sql("""
SELECT 
    row_number() OVER (
        PARTITION BY '' 
        ORDER BY '' 
    ) as pos,
    *
FROM 
    SONGS_INFO
""")

songs_info_df = songs_info_df.sort("track_uri")

songs_df = songs_info_df.select("pos", "track_uri")

RATING_VECTOR_LENGTH = songs_df.count()

In [19]:
# songs_info_df.show()
# songs_df.show()

+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+
|   pos|         artist_name|           track_uri|          artist_uri|          track_name|           album_uri|duration_ms|          album_name|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+
|154210| Two Steps from Hell|spotify:track:000...|spotify:artist:2q...|           Nightwood|spotify:album:1BD...|     189638|Colin Frake On Fi...|
|354525|         Little Simz|spotify:track:000...|spotify:artist:6e...|Mandarin Oranges ...|spotify:album:32R...|     198000|             E.D.G.E|
|459225|           Celly Cel|spotify:track:000...|spotify:artist:6k...|         Shady World|spotify:album:6LO...|     155238|      The Lost Tapes|
|659120|El Poder De Zacat...|spotify:track:000...|spotify:artist:3n...|         Abeja Miope|spotify:album:18X...|     

Preprocessing the dataframe in order to associate to each track_uri an integer, that will represent the position of the track in the rating_vector. This is useful in order to avoid doing a lot of joins when generating the rating_vectors.

In [20]:
# def map_track_df_to_pos(playlist_df: DataFrame, mapping: DataFrame) -> List[DataFrame]:
#   songs_df_list = [get_all_songs(spark.createDataFrame([row])) for row in tqdm(slice_df.collect(), desc="Creating list of dataframes")]
#   track_uri_to_id = songs_df.select('track_uri', 'pos').rdd.collectAsMap()
#   track_uri_to_id_udf = udf(lambda x: track_uri_to_id.get(x), IntegerType())
#   songs_df_mapped_list = []

#   for df in tqdm(songs_df_list, desc="Mapping uris to pos"):
#       df = df.withColumn('pos', track_uri_to_id_udf(col('track_uri')))
#       songs_df_mapped_list.append(df)
  
#   return songs_df_mapped_list

# songs_df_mapped_list = map_track_df_to_pos(slice_df, songs_df)

In [273]:
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, ArrayType

def map_track_df_to_pos(playlist_df: DataFrame, mapping: DataFrame) -> DataFrame:
    track_uri_to_id = songs_df.select('track_uri', 'pos').rdd.collectAsMap()

    track_uri_to_id_udf = udf(lambda x: track_uri_to_id.get(x), IntegerType())

    # Define a UDF to map track URIs to integer IDs within each element of the list
    
    def extract_vector(tracks):
      pos_list = set()
      for row in tracks:
        uri = track_uri_to_id.get(row.track_uri)
        pos_list.add(uri)

      return SparseVector(RATING_VECTOR_LENGTH + 1, sorted(list(pos_list)), [1 for _ in pos_list])

    map_track_uri_udf = udf(lambda tracks: extract_vector(tracks), returnType=VectorUDT())

    # Apply the mapping UDF on the "tracks" column of the slice_df dataframe
    mapped_df = playlist_df.withColumn('tracks', map_track_uri_udf(col('tracks')))

    return mapped_df

if not os.path.exists(PLAYLIST_EMBEDDINGS):
  mapped_slice_df = map_track_df_to_pos(slice_df, songs_df)
  mapped_slice_df.write.parquet(PLAYLIST_EMBEDDINGS)
else:
  mapped_slice_df = spark.read.parquet(PLAYLIST_EMBEDDINGS)


In [22]:
#@title Old way of creating the rating vector dataframe
# def _create_rating_df(playlist_row: Row, songs_df: DataFrame) -> DataFrame:
#   """
#   Creates a dataframe that represents the "ratings" for a playlist in the dataframe
#   """
#   playlist_row = spark.createDataFrame([playlist_row], playlist_schema)
#   playlist_uris = get_all_songs(playlist_row)

#   joined = songs_df.join(playlist_uris, on="track_uri", how="right")
#   return joined


# def _check_songs_ordering(playlist_row: DataFrame, songs_df: DataFrame) -> bool:
#   """
#   Returns a boolean that indicates if the ordering in the songs_df and rating_df is the same
#   """
#   playlist_row = spark.createDataFrame([playlist_row], playlist_schema)
#   playlist_uris = get_all_songs(playlist_row, True).withColumnRenamed("in_playlist", "isin")

#   joined = songs_df.join(playlist_uris, on="track_uri", how="right")
#   joined_left = songs_df.join(playlist_uris, on="track_uri", how="left").filter("isin == 1")
#   assert joined.collect() == joined_left.collect(), f"The order of songs_df is different from the order of rating_df!"

# # def _extract_rating_vector(rating_df: DataFrame) -> SparseVector:
# #   """
# #   Extracts the rating vectors for each playlist 
# #   """
# #   dense_vector = DenseVector([row.isin for row in rating_df.select("isin").collect()])
# #   return dense_to_sparse(dense_vector)

# def _extrac_sparse_rating_vector(rating_df: DataFrame) -> SparseVector:
#   indices = np.sort([row.pos for row in rating_df.collect()])
#   return SparseVector(RATING_VECTOR_LENGTH, indices, np.ones(indices.shape[0]) )

# def rating_vector_from_row(playlist_row: Row, songs_df: DataFrame):
#   """
#   Pipelines togheter create_rating_df and extract_rating_vector.
#   """
#   rating_df_1 = _create_rating_df(playlist_row, songs_df)
#   rating_vector_1 = _extrac_sparse_rating_vector(rating_df_1)
#   return rating_vector_1

# # t1 = time.time() 

# # rating_vector_1 = rating_vector_from_row(slice_df.first(), songs_df)

# # t2 = time.time()

# # t2 - t1, rating_vector_1, type(rating_vector_1)

In [23]:
def jaccard_similarity(vector_1: SparseVector, vector_2: SparseVector) -> float:
  # Convert SparseVectors to sets
  set1 = set(vector_1.indices)
  set2 = set(vector_2.indices)

  # Calculate the intersection and union of the sets
  intersection = len(set1.intersection(set2))
  union = len(set1.union(set2))

  # Calculate the similarity
  similarity = intersection / union

  return similarity

In [24]:
#@title Old way of creating the ratinf vector dataframe pt.2
# def create_rating_vectors_df(playlists_df: DataFrame) -> DataFrame:
#   rating_vectors = []

#   for playlist_row in tqdm(playlists_df.collect(), desc="Creating rating vectors"):
#     rating_vector = rating_vector_from_row(playlist_row, songs_df)
#     new_row = Row(playlist_id=playlist_row.pid, rating_vector=rating_vector)
#     rating_vectors.append([new_row])
#   return spark.createDataFrame(rating_vectors)

# if os.path.exists(RATING_VECTOR_FILE_PATH):
#   # rv_schema = StructType([StructField('playlist_id', LongType(), True), StructField('rating_vector', pyspark.ml.linalg.VectorUDT(), True)])
#   rating_vectors_df = spark.read.parquet(RATING_VECTOR_FILE_PATH)
#   rv_df = rating_vectors_df.select(col("_1.playlist_id").alias("playlist_id"), col("_1.rating_vector").alias("rating_vector"))
# else:
#   rating_vectors_df = create_rating_vectors_df(slice_df)
#   rating_vectors_df.write.parquet(RATING_VECTOR_FILE_PATH)

Creating a function that gets in input the playlist to continue, and returns a Dataframe that indicates its similarity with each other playlist in the dataset.

In [25]:
mapped_slice_df.first()

Row(name='disney', collaborative='false', pid=1000, modified_at=1457827200, num_tracks=189, num_albums=16, num_followers=1, tracks=SparseVector(681806, {127: 1.0, 1904: 1.0, 2439: 1.0, 3437: 1.0, 4136: 1.0, 4941: 1.0, 6237: 1.0, 6362: 1.0, 8263: 1.0, 8606: 1.0, 9079: 1.0, 9696: 1.0, 9998: 1.0, 10676: 1.0, 10958: 1.0, 12712: 1.0, 13117: 1.0, 14708: 1.0, 15578: 1.0, 16236: 1.0, 16384: 1.0, 17407: 1.0, 17587: 1.0, 17970: 1.0, 18322: 1.0, 18395: 1.0, 19986: 1.0, 21024: 1.0, 21275: 1.0, 21652: 1.0, 22516: 1.0, 22918: 1.0, 22988: 1.0, 23837: 1.0, 24693: 1.0, 24803: 1.0, 24964: 1.0, 25323: 1.0, 26812: 1.0, 27807: 1.0, 31090: 1.0, 31091: 1.0, 31371: 1.0, 32346: 1.0, 32580: 1.0, 33508: 1.0, 33621: 1.0, 33761: 1.0, 35472: 1.0, 35787: 1.0, 35947: 1.0, 36552: 1.0, 36602: 1.0, 36745: 1.0, 36746: 1.0, 37777: 1.0, 39611: 1.0, 51029: 1.0, 54561: 1.0, 59119: 1.0, 61879: 1.0, 83930: 1.0, 89372: 1.0, 95237: 1.0, 97166: 1.0, 99267: 1.0, 99594: 1.0, 100077: 1.0, 102061: 1.0, 102527: 1.0, 103131: 1.0, 10556

In [281]:
def create_similarity_df(input_vector: DataFrame, rating_vectors_df: DataFrame, similarityFunction: Callable) -> DataFrame:
  rv_df_input = rating_vectors_df.crossJoin(input_vector)
  similarity_udf = udf(similarityFunction, returnType='double')
  result_df = rv_df_input.withColumn("similarity", similarity_udf(rv_df_input["input_vector"], rv_df_input["rating_vector"]))
  return result_df

rv_df = mapped_slice_df.withColumnRenamed("tracks", "rating_vector")
#TODO: Just for test, we take the first playlist as the playlist to be continued
first_playlist_vector = rv_df.limit(1).select("rating_vector").withColumnRenamed("rating_vector","input_vector")
result_df = create_similarity_df(first_playlist_vector, rv_df, jaccard_similarity)

Curse of dimensionality! We can see that each playlist is very dissimilar from each other playlist.

If we filter the playlists that have a strictly positive similarity with the input playlist, and order them by descending similarity, we can see that the name (that we assume is very informative for the content of the playlist) is very similar, meaning that the algorithm seems to work!

In [27]:
result_df.filter("similarity > 0").orderBy(col("similarity").desc()).show()

+---------------+-------------+-----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+--------------------+-------------------+
|           name|collaborative|  pid|modified_at|num_tracks|num_albums|num_followers|       rating_vector|num_edits|duration_ms|num_artists|        input_vector|         similarity|
+---------------+-------------+-----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+--------------------+-------------------+
|         disney|        false| 1000| 1457827200|       189|        16|            1|(681806,[127,1904...|        4|   31428282|         65|(681806,[127,1904...|                1.0|
|         Disney|        false|34420| 1430784000|       179|        15|            1|(681806,[255,1904...|        3|   25533248|         54|(681806,[127,1904...|0.22635135135135134|
|        Disney✨|        false|12990| 1450915200|       113|        32|            1|(6818

Now, in order to suggest some songs to continuate the input playlist, let's thake the. $K$ top most similar playlists

In [41]:
def get_top_k_results(playlist_pid: int, similarity_df: DataFrame, k: int = 20) -> DataFrame:
  return similarity_df.filter( (col('pid') != playlist_pid)).orderBy(col("similarity").desc()).limit(k)

first_playlist_pid = rv_df.limit(1).select("pid").first().pid
top_k_results = get_top_k_results(first_playlist_pid, result_df)

In [42]:
top_k_results.show()

+---------------+-------------+-----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+--------------------+-------------------+
|           name|collaborative|  pid|modified_at|num_tracks|num_albums|num_followers|       rating_vector|num_edits|duration_ms|num_artists|        input_vector|         similarity|
+---------------+-------------+-----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+--------------------+-------------------+
|         Disney|        false|34420| 1430784000|       179|        15|            1|(681806,[255,1904...|        3|   25533248|         54|(681806,[127,1904...|0.22635135135135134|
|        Disney✨|        false|12990| 1450915200|       113|        32|            1|(681806,[1791,190...|        3|   20665774|         64|(681806,[127,1904...|0.18775510204081633|
|         Disney|        false| 3323| 1391817600|       133|        20|            1|(6818

Making a prediction

In [30]:
# from pyspark.ml.linalg import VectorUDT

# def add_sparse_vectors(accumulator: SparseVector, vector: SparseVector, weight: float) -> SparseVector:
#     accumulator_vec = accumulator.toArray() 
#     array_2 = vector.toArray() * weight

#     summed_array = accumulator_vec + array_2

#     values = [value for value in summed_array if value != 0]
#     sorted_indices = [index for index, value in enumerate(summed_array) if value != 0]
#     return SparseVector(accumulator_vec.size, sorted_indices, values)

# @udf(returnType=VectorUDT())
# def accumulate_sparse_vectors(accumulator, rating_vector, similarity):
#     summed_vector = add_sparse_vectors(accumulator, rating_vector, similarity)
#     return summed_vector

# df = top_k_results.withColumn('accumulated_vector', accumulate_sparse_vectors(top_k_results["rating_vector"], top_k_results["rating_vector"], top_k_results['similarity']))

We want to obtain a single embedding for all the $K$ top most similar playlists, that will be the rating vector. We can then pick the indices of the $n$ top greatest values form this vector, and those will be the $n$ songs that we will reccomend.

In order to aggregate the $k$ embeddings into a single one, I decided to take an average, weighted by the similarity value.

In [103]:
#TODO: exclude the songs that already are in the playlist
def accumulate_top_k_results(top_k_results: DataFrame):
  accumulator = np.zeros(RATING_VECTOR_LENGTH + 1)
  similarity_sum = 0
  #TODO: maybe do it with a rdd.map to be more performant
  top_k = top_k_results.collect()
  for row in top_k:
    accumulator += row.rating_vector.toArray() * row.similarity
    similarity_sum += row.similarity
  
  accumulator /= similarity_sum
  values = [value for value in accumulator if value != 0]
  sorted_indices = [index for index, value in enumerate(accumulator) if value != 0]
  return SparseVector(accumulator.size, sorted_indices, values)

accumulated_vector = accumulate_top_k_results(top_k_results)

In [32]:
# @udf(returnType="int")
# def compute_vector_len(rating_vector):
#     return len(rating_vector.indices)
# final_df = df.withColumn('accumulated_vector_len', compute_vector_len(df["rating_vector"]))

In [ ]:
# accumulated_vector = final_df.orderBy(col("accumulated_vector_len").desc()).first().accumulated_vector

In [105]:
def get_top_n_values(vector: SparseVector, n:int = 10) -> List[Tuple[int, float]]:
  elements = list(enumerate(vector.toArray()))
  sorted_elements = sorted(elements, key=lambda x: x[1], reverse=True)
  top_n_indices = [(index, confidence) for index, confidence in sorted_elements[:n]]
  return top_n_indices

top_n_reccomendations = get_top_n_values(accumulated_vector)

In [106]:
top_n_reccomendations

[(364737, 0.866605950361276),
 (18322, 0.8485493061139573),
 (343633, 0.8485493061139573),
 (373510, 0.8485493061139573),
 (344041, 0.84513808582077),
 (36962, 0.8245530907321501),
 (377909, 0.8108159417687045),
 (21652, 0.8095907772059491),
 (10958, 0.7976130554051545),
 (344569, 0.7976130554051545)]

In [55]:
def convert_df_to_map(df: DataFrame) -> DataFrame:
  return df\
  .withColumn('aggregated', \
              to_json(
                  struct(
                      "artist_name", 
                      "track_uri", 
                      "artist_uri", 
                      "track_name", 
                      "album_uri", 
                      "duration_ms", 
                      "album_name")
                  )
              ).select("pos", "aggregated")
              
songs_info_map = convert_df_to_map(songs_info_df)

In [70]:
prediction_schema = StructType([
     StructField('artist_name', StringType(), True),
     StructField('track_uri', StringType(), True),
     StructField('artist_uri', StringType(), True),
     StructField('track_name', StringType(), True),
     StructField('album_uri', StringType(), True),
     StructField('duration_ms', LongType(), True),
     StructField('album_name', StringType(), True),
    StructField('confidence', FloatType(), True),
     ])

In [119]:
import json
#TODO: For now this works, but it's very slow, and since this has to be executed online,
# consider to directly embed the song information inside the dataframe when computing the songs
# to recommend.
def recommendation_song_info(recommendation: List[Tuple[int, float]], songs_info_map) -> Row:
  result = []
  for index, confidence in tqdm(recommendation, desc="Extracting song info from recommendations..."):
    song_info_dict = json.loads(songs_info_map.lookup(index)[0])
    song_info_dict.update({"confidence":float(confidence)})
    result.append(Row(**song_info_dict))
  return spark.createDataFrame(result, schema=prediction_schema)

# songs_info = recommendation_song_info(top_n_reccomendations, songs_info_map.rdd)

### Putting it all togheter
We now define a single function that will get a playlist in input and will reccomend $n$ songs.

In [292]:
#TODO: this now takes a playlist and extracts its PID, if a playlist is built from scratch the PID shouldn't be defined
# A solution would be to pass the playlist row with the PID = Nan and then have a condition when extracting the PID. If Nan, ignore it
def user_based_recommendation(playlist: DataFrame, mapped_slice_df: DataFrame, similarity_function: Callable, n:int = 50) -> DataFrame:
  with tqdm(total=5, desc="Generating recommendations...") as pbar:
    rv_df = mapped_slice_df.withColumnRenamed("tracks", "rating_vector") #TODO: Parse the rv_df before and then remove this
    
    #TODO: define the songs_df as input to the function
    playlist_vector = map_track_df_to_pos(playlist, songs_df).select("tracks").withColumnRenamed("tracks", "input_vector")
    similarity_df = create_similarity_df(playlist_vector, rv_df, jaccard_similarity)
    pbar.update(1)

    top_k_results = get_top_k_results(playlist.first().pid, similarity_df)
    pbar.update(1)

    accumulated_vector = accumulate_top_k_results(top_k_results)
    pbar.update(1)

    top_n_indices = get_top_n_values(accumulated_vector, n=n)
    pbar.update(1)

    recommended_songs_info = recommendation_song_info(top_n_indices, songs_info_map.rdd)
    pbar.update(1)

  return recommended_songs_info

# playlist = slice_df.filter("pid == 1010").limit(1)
# final_recommendation = user_based_recommendation(playlist, mapped_slice_df, jaccard_similarity, n=5)

## Performance Evaluation

Let's load the test dataset, where there are playlists that have missing songs

In [254]:
# TEST_PLAYLISTS = os.path.join(GDRIVE_DATA_DIR, "challenge_set.json")
# test_playlist_schema = StructType([
#     StructField("name", StringType(), True),
#     StructField("num_holdouts", IntegerType(), True),
#     StructField("pid", IntegerType(), True),
#     StructField("num_tracks", IntegerType(), True),
#     StructField("tracks", ArrayType(song_schema), True),
#     StructField("num_samples", IntegerType(), True)
# ])
# test_playlists = spark.read.schema(test_playlist_schema).json(TEST_PLAYLISTS, multiLine=True)

Since we are not addressing the cold start problem, we will filter out all those playlists that do not contain any song

In [255]:
# test_playlists.show()

+--------------------+------------+----+----------+------+-----------+
|                name|num_holdouts| pid|num_tracks|tracks|num_samples|
+--------------------+------------+----+----------+------+-----------+
|build/challenge/c...|        null|null|      null|  null|       null|
+--------------------+------------+----+----------+------+-----------+



In [265]:
def train_test_split(playlist: DataFrame):
    songs = playlist.select(explode("tracks")).select("col.*")
    train_df, test_df = songs.randomSplit([0.8, 0.2], seed=42)

    row = playlist.collect()[0]

    playlist_train = spark.createDataFrame(
        [Row(
            name=row.name,
            collaborative=row.collaborative,
            pid=row.pid,
            modified_at=row.modified_at,
            num_tracks=row.num_tracks,
            num_albums=row.num_albums,
            num_followers=row.num_followers,
            tracks=train_df.collect(),
            num_edits=row.num_edits,
            duration_ms=row.duration_ms,
            num_artists=row.num_artists,
        )],
        schema=playlist_schema,
    )

    playlist_test = spark.createDataFrame(
        [Row(
            name=row.name,
            collaborative=row.collaborative,
            pid=row.pid,
            modified_at=row.modified_at,
            num_tracks=row.num_tracks,
            num_albums=row.num_albums,
            num_followers=row.num_followers,
            tracks=test_df.collect(),
            num_edits=row.num_edits,
            duration_ms=row.duration_ms,
            num_artists=row.num_artists,
        )],
        schema=playlist_schema,
    )
    
    return playlist_train, playlist_test


In [297]:
playlist_to_recommend = slice_df.filter("pid == 1000")
playlist_train, playlist_test = train_test_split(playlist_to_recommend)

In [132]:
def precision_at_k(recommendations, ground_truth):
    """
    Calculates precision at k for the recommendations.
    """
    total_relevant = ground_truth.count()
    recommended_relevant_tracks = recommendations.join(ground_truth, "track_uri").count()
    # recommended_relevant_artist = recommendations.join(ground_truth, "track_uri").count()

    precision = recommended_relevant_tracks / float(recommendations.count())
    return precision

In [298]:
ground_truth = playlist_test.select(explode("tracks")).select("col.*")

Let's generate some recommendation for the train playlist

In [ ]:
# playlist_train.show()

In [ ]:
recommendations = user_based_recommendation(playlist_to_recommend, mapped_slice_df, jaccard_similarity, n=20)

Generating recommendations...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
recommendations.show()

In [ ]:
precision = precision_at_k(recommendations, ground_truth)

In [ ]:
precision

# Fighting against the curse of dimensionality: Matrix Factorization

We want to define $\mathbf{x}_u \in \mathbb{R}^d$ $d$-dimensional vector that represents the user $u$, and $\mathbf{w}_i \in \mathbb{R}^d$ vector that represent the item $i$.

We then can estimate the rating of user $u$ for the item $i$ by computing
\begin{equation}
\hat{r}_{u, i}=\mathbf{x}_u^T \cdot \mathbf{w}_i=\sum_{j=1}^d x_{u, j} w_{j, i}
\end{equation}
Or, in matrix notation,

\begin{equation}
\underbrace{R}_{m \times n} =
\underbrace{X}_{m \times d}
\underbrace{W^T}_{d \times n}
\end{equation}

### How to learn $X$ and $W$
The matrix $R$ is partially known and filled with the observations inside the dataset $\mathcal{D}$. In order to learn the latent factor representations $X$ and $W$, we minimize the following loss function:
\begin{equation}
L(X, W)=\sum_{(u, i) \in \mathcal{D}}\underbrace{\left(r_{u, i}-\mathbf{x}_u^T \cdot \mathbf{w}_i\right)^2}_{\text{squared error term}}+\underbrace{\lambda\left(\sum_{u \in \mathcal{D}}\left\|\mathbf{x}_u\right\|^2+\sum_{i \in \mathcal{D}}\left\|\mathbf{w}_i\right\|^2\right)}_{\text{regularization term}}
\end{equation}

We can then minimize the loss using Stochastic Gradient Descent or Alternating Least Squares.

# Matrix Factorization
Generate a matrix Y where each column represent a playlist and each row represent a song, the (i,j) entry will be 1 if the playlist contains the song, 0 otherwise.

In [ ]:
# Throw error in order to not execute the following code
raise ValueError()

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.functions import explode
spark.conf.set("spark.sql.pivotMaxValues", 1000000)

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
# Get a DataFrame of only the relevant columns from the playlist schema
# playlists = slice_df.select("pid", "tracks.track_uri")
# playlists = playlists.select("pid", explode("track_uri").alias("song_uri"))
# playlists = playlists.withColumn("song_id", dense_rank().over(Window.orderBy("song_uri")))
# plaulists = playlists.withColumn("rating", lit(1))
# playlists.count()

from pyspark.sql.functions import expr


In [ ]:
from pyspark.sql.functions import explode
import random
tracks_df = slice_df.select("pid", explode("tracks").alias("track")).select("pid", "track.track_uri")
tracks_df = tracks_df.withColumn("rating", lit(1))
# tracks_df = tracks_df.withColumn("rating", (rand() * 10 + 1).cast("integer"))

In [ ]:
tracks_df.show()

In [ ]:
# # Explode the tracks array column into multiple rows
# # tracks_df = slice_df.select("pid", explode("tracks").alias("track"))
# # tracks_df = slice_df.select("pid", "tracks", "tracks")
# tracks_df = slice_df.select("pid", explode("tracks").alias("track")).select("pid", "track.track_uri", "track.pos")

# # Select relevant columns and add a rating column with value 1
# playlist_track_df = tracks_df.withColumn("rating", lit(1))

# # Get distinct track_uri values and join with playlist_track_df
# all_tracks_df = slice_df.select(explode("tracks").alias("track")).select("track.track_uri").distinct()
# all_playlists_df = slice_df.select("pid").distinct()

# all_against_all = all_tracks_df.join(all_playlists_df).distinct()

# from pyspark.sql.functions import when, col

# # playlist_track_rating_df = playlist_track_df.join(all_against_all, ["pid", "track_uri"], "left_outer") \
# #     .withColumn("rating", when(col("pos").isNull(), 0).otherwise(1))

# playlist_track_rating_df = all_against_all.join(playlist_track_df, ["pid", "track_uri"], "left_outer") \
#     .withColumn("rating", when(col("pos").isNull(), 0).otherwise(1)) \
#     .drop("pos")


In [ ]:
playlist_track_rating_df = tracks_df.withColumn("song_id", dense_rank().over(Window.orderBy("track_uri")))

In [ ]:
playlist_track_rating_df.show(truncate=False)

In [ ]:
als = ALS(userCol="pid", itemCol="song_id", ratingCol="rating", nonnegative=True, coldStartStrategy="drop")

In [ ]:
from typing import Tuple
import random

def train_test_split(df: DataFrame, split_ratio: float, seed: Optional[int] = None) -> Tuple[DataFrame, DataFrame]:
  random.seed(seed)
  distinct_pids = df.select("pid").distinct().rdd.map(lambda x: x[0]).collect()
  random.shuffle(distinct_pids)
  split_index = int(len(distinct_pids) * split_ratio)
  train_pids = distinct_pids[:split_index]
  test_pids = distinct_pids[split_index:]
  train_df = df.filter(col("pid").isin(train_pids))
  test_df = df.filter(col("pid").isin(test_pids))
  return train_df, test_df



In [ ]:
training, test = playlist_track_rating_df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
model = als.fit(training)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [ ]:
predictions.filter(col("prediction") != "NaN").count(), predictions.filter(col("prediction") == "NaN").count()

In [ ]:
rmse

In [ ]:
subset = playlist_track_rating_df.select("pid").distinct().limit(1)
subUserRecs = model.recommendForUserSubset(subset, 10)

In [ ]:
subset.show()

In [ ]:
subUserRecs.show(truncate=False)

In [ ]:
def song_name_from_id(song_id: int, reverse_lookup: DataFrame) -> str:
  return 
  
def interpretRecommendation(recommended_result: DataFrame) -> str:
  return

In [ ]:
userRecs = model.recommendForAllUsers(1).orderBy("recommendations")
userRecs.show(truncate=False)
userRecs.count()

In [ ]:
slice_df.filter(col("pid") == 1710).select(explode("tracks.track_name")).show()

In [ ]:
track_uris = playlist_track_rating_df.filter(col("song_id") == 588).select("track_uri")
track_uris.first()